# Intro

# Imports

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import time
from pathlib import Path
import json
import sys
from importlib import reload
sys.path.append('../src')

# Routines

In [2]:
def sleep(duration):
    print(f'sleep {duration}s')
    time.sleep(duration)
    
def parse(text, parser='html.parser'):
    return BeautifulSoup(text, parser)

def fetch(url, file_path=None, session=None, max_attempts=10, custom_headers=None):
    if custom_headers is None:
        custom_headers = {}
    
    if file_path is not None:
        print(f'Fetch {url} to {file_path}')
        if Path(file_path).exists():
            print('Already exists. Skipped.')
            return None
    else: 
        print(f'Fetch {url}')
    
    for attempt in range(max_attempts):
        if session is not None:
            response = session.get(url, headers=custom_headers)
        else:
            response = get(url, headers=custom_headers)
        
        # handle HTTP 429 Too Many Requests
        if response.status_code == 429:
            delay = (attempt + 1) * 10
            print(f'Retrying in {delay} seconds')
            time.sleep(delay)
        else:   
            break    
    
    response.raise_for_status()
    
    if session is not None:
        print(f'Session cookies: {len(session.cookies)}')
        
    if file_path is not None:
        with open(file_path, 'wb') as file:
            file.write(response.content)
        
    return response.text

In [25]:
from utilities import excel_reader
df = excel_reader.ExcelReader().read_df_from_excel("C4W_dataset_from_the_DEC_pdf.xlsx")

Read file C4W_dataset_from_the_DEC_pdf.xlsx: 0.94s
Processed file C4W_dataset_from_the_DEC_pdf.xlsx: 0.79s


In [27]:
import requests
import os

folder_to_save = "../tmp/usaid_additional_files"
for i in range(len(df)):
    if df["pdf_link"].values[i].strip():
        print(i, df["pdf_link"].values[i].strip())
        url = df["pdf_link"].values[i].strip()
        filename = os.path.basename(url)
        if os.path.exists(os.path.join(folder_to_save, filename)):
            continue
        r = requests.get(url, stream=True)
        chunk_size = 2000

        with open(os.path.join(folder_to_save, filename), 'wb') as fd:
            for chunk in r.iter_content(chunk_size):
                fd.write(chunk)

0 https://pdf.usaid.gov/pdf_docs/PNABA262.pdf
4 https://pdf.usaid.gov/pdf_docs/PNAAL471.pdf
6 https://pdf.usaid.gov/pdf_docs/PA00W6RQ.pdf
8 https://pdf.usaid.gov/pdf_docs/PA00W6J7.pdf
13 https://pdf.usaid.gov/pdf_docs/PA00WH9Z.pdf
25 https://pdf.usaid.gov/pdf_docs/PDAAS009.pdf
26 https://pdf.usaid.gov/pdf_docs/PA00W76F.pdf
27 https://pdf.usaid.gov/pdf_docs/PA00VPJP.pdf
28 https://pdf.usaid.gov/pdf_docs/PNABG801.pdf
29 https://pdf.usaid.gov/pdf_docs/PNABS812.pdf
31 https://pdf.usaid.gov/pdf_docs/PA00TFZZ.pdf
60 https://pdf.usaid.gov/pdf_docs/PNAED176.pdf
62 https://pdf.usaid.gov/pdf_docs/PDAAX130.pdf
63 https://pdf.usaid.gov/pdf_docs/PNAAU179.pdf
64 https://pdf.usaid.gov/pdf_docs/PDAAP519.pdf
65 https://pdf.usaid.gov/pdf_docs/PNAAW174.pdf
69 https://pdf.usaid.gov/pdf_docs/PNAAW701.pdf
96 https://pdf.usaid.gov/pdf_docs/PDABF389.pdf
122 https://pdf.usaid.gov/pdf_docs/PA00TM7M.pdf
125 https://pdf.usaid.gov/pdf_docs/PDAAL550.pdf
126 https://pdf.usaid.gov/pdf_docs/PNAAK981.pdf
136 https://pd

850 https://pdf.usaid.gov/pdf_docs/PNAAZ556.pdf
854 https://pdf.usaid.gov/pdf_docs/PNABE247.pdf
872 https://pdf.usaid.gov/pdf_docs/PNAAY001.pdf
875 https://pdf.usaid.gov/pdf_docs/PNAAG476.pdf
898 https://pdf.usaid.gov/pdf_docs/PNAAU412.pdf
901 https://pdf.usaid.gov/pdf_docs/PNAAV913.pdf
915 https://pdf.usaid.gov/pdf_docs/PDAAV056.pdf
918 https://pdf.usaid.gov/pdf_docs/PNAAW792.pdf
920 https://pdf.usaid.gov/pdf_docs/PDAAK283.pdf
921 https://pdf.usaid.gov/pdf_docs/PDAAV612.pdf
923 https://pdf.usaid.gov/pdf_docs/PNAAJ844.pdf
925 https://pdf.usaid.gov/pdf_docs/PNAAZ324.pdf
926 https://pdf.usaid.gov/pdf_docs/PNAAZ325.pdf
930 https://pdf.usaid.gov/pdf_docs/PDAAX663.pdf
933 https://pdf.usaid.gov/pdf_docs/PDABD925.pdf
939 https://pdf.usaid.gov/pdf_docs/PNAAW195.pdf
941 https://pdf.usaid.gov/pdf_docs/PNABC435.pdf
951 https://pdf.usaid.gov/pdf_docs/PNABB231.pdf
954 https://pdf.usaid.gov/pdf_docs/PNABB236.pdf
960 https://pdf.usaid.gov/pdf_docs/PNAAR444.pdf
981 https://pdf.usaid.gov/pdf_docs/PDABG

In [ ]:
import requests

folder_to_save = "../tmp/usaid_pdfs"
for i in range(len(df)):
    if df["File"].values[i].strip():
        print(i, df["File"].values[i].strip())
        url = df["File"].values[i].strip()
        filename = os.path.basename(url)
        if os.path.exists(os.path.join(folder_to_save, filename)):
            continue
        r = requests.get(url, stream=True)
        chunk_size = 2000

        with open(os.path.join(folder_to_save, filename), 'wb') as fd:
            for chunk in r.iter_content(chunk_size):
                fd.write(chunk)

In [24]:
with open("file_extensions.txt", "w") as f:
    for word in [a.text[1:].lower() for a in doc.select("tr > td > a")]:
        if word.strip() != "":
            f.write(word + "\n")

In [37]:
doc = parse(fetch("https://devtracker.dfid.gov.uk/projects/GB-GOV-1-300532"))

Fetch https://devtracker.dfid.gov.uk/projects/GB-GOV-1-300532


In [9]:
[ "https://devtracker.dfid.gov.uk" + item["href"] for item in doc.select('div.search-result a')]

['https://devtracker.dfid.gov.uk/projects/GB-COH-04105827-AIDDIRECT',
 'https://devtracker.dfid.gov.uk/projects/GB-GOV-50-PF-06-DIG',
 'https://devtracker.dfid.gov.uk/projects/GB-GOV-50-PF-15-MEX',
 'https://devtracker.dfid.gov.uk/projects/GB-1-204338',
 'https://devtracker.dfid.gov.uk/projects/GB-1-204963',
 'https://devtracker.dfid.gov.uk/projects/us-ein-522044704-NewVaccinesForTB',
 'https://devtracker.dfid.gov.uk/projects/XM-DAC-41140-SS01.08.041.CPA5',
 'https://devtracker.dfid.gov.uk/projects/GB-1-205181',
 'https://devtracker.dfid.gov.uk/projects/GB-GOV-1-300644',
 'https://devtracker.dfid.gov.uk/projects/GB-1-205288',
 'https://devtracker.dfid.gov.uk/projects/XM-DAC-41140-NP02.02.021.SMP1',
 'https://devtracker.dfid.gov.uk/projects/GB-GOV-12-00003',
 'https://devtracker.dfid.gov.uk/projects/XM-DAC-41140-CI02.01.011.SMP1',
 'https://devtracker.dfid.gov.uk/projects/GB-GOV-1-300727',
 'https://devtracker.dfid.gov.uk/projects/GB-GOV-1-300815',
 'https://devtracker.dfid.gov.uk/proje

# AGRIS

In [13]:
txt = fetch(
    r'http://agris.fao.org/agris-search/biblio.action?agrovocString=&agrovocToAdd=&agrovocToRemove=&advQuery=&centerString=&centerToRemove=&onlyFullText=false&filterString=&filterToRemove=&typeString=&typeToRemove=&filterQuery=environment+AND+incentive+AND+farmer&method%3Aexecute=&operator=Required&field=0&typeresultsField=Both&fromDate=0&toDate=0&center=0&lang=0&typeToAdd=0&enableField=Disable&aggregatorField=Disable'
)

Fetch http://agris.fao.org/agris-search/biblio.action?agrovocString=&agrovocToAdd=&agrovocToRemove=&advQuery=&centerString=&centerToRemove=&onlyFullText=false&filterString=&filterToRemove=&typeString=&typeToRemove=&filterQuery=environment+AND+incentive+AND+farmer&method%3Aexecute=&operator=Required&field=0&typeresultsField=Both&fromDate=0&toDate=0&center=0&lang=0&typeToAdd=0&enableField=Disable&aggregatorField=Disable


In [14]:
doc = parse(txt)

In [21]:
for m in matches:
    print(m.text)
    article_id = m['href'].split('=')[-1]  
    href = 'http://agris.fao.org/agris-search/' + article_id
    meta_href = 'http://agris.fao.org/agris-search/export!endNote.action?arn=' + article_id
    
    print(article_id)
    print(href)
    print(meta_href)
    print()

Priorities for the Development of Smallholder Agriculture in Swaziland
US2014600446
http://agris.fao.org/agris-search/US2014600446
http://agris.fao.org/agris-search/export!endNote.action?arn=US2014600446

Early Insights from Financial Diaries of Smallholder Households
US2015601175
http://agris.fao.org/agris-search/US2015601175
http://agris.fao.org/agris-search/export!endNote.action?arn=US2015601175

Economic assessment of the beef cattle commodity chain for smallholder and large farmers in East Java, Indonesia
PH2016000043
http://agris.fao.org/agris-search/PH2016000043
http://agris.fao.org/agris-search/export!endNote.action?arn=PH2016000043

Zambia - Commercial Value Chains in Zambian Agriculture : Do Smallholders Benefit?
US2014600223
http://agris.fao.org/agris-search/US2014600223
http://agris.fao.org/agris-search/export!endNote.action?arn=US2014600223

Smallholder farmers’ perceptions of climate change and conservation agriculture: evidence from Zambia
GB2013201769
http://agris.fao.o

In [160]:
import re
import sys

sys.path.append('../src')
from webscrapping import webcrawler_agris_xml
from webscrapping import webcrawler_base
webcrawler_agris_xml = reload(webcrawler_agris_xml)

_webcrawler_unep = webcrawler_agris_xml.WebCrawlerAGRIS_XML()
_webcrawler_unep.crawl_query('http://agris.fao.org/agris-search/biblio.action?startIndexSearch=0&typeToRemove=journal_article&advQuery=smallholder&agrovocString=&centerString=&onlyFullText=false&filterString=%2BpublicationDate%3A%5B2008+TO+2018%5D&typeString=journal_article&enableField=Disable&aggregatorField=Disable', "../tmp/AGRIS_XML")

In [161]:
_webcrawler_unep.prepare_dataset("../tmp/AGRIS", "../tmp/Datasets/AGRIS.xlsx")

In [13]:
from webscrapping import webcrawler_agris
from webscrapping import webcrawler_base
from webscrapping import webcrawler_logger
from importlib import reload
webcrawler_base = reload(webcrawler_base)
webcrawler_agris = reload(webcrawler_agris)
webcrawler_logger = reload(webcrawler_logger)

_webcrawler_unep = webcrawler_agris.WebCrawlerAGRIS()
#_webcrawler_unep.crawl_query('http://agris.fao.org/agris-search/searchIndex.do?query=agriculture+%2B+farming+%2B+income', '/tmp/agris_try')
_webcrawler_unep.crawl_query('http://agris.fao.org/agris-search/searchIndex.do?query=%22farmer%27s%20organization%22&typeToAdd=journal_article&startIndexSearch=0', "../tmp/AGRIS_team_2_check")



Page 0
Fetch http://agris.fao.org/agris-search/biblio.action?advQuery=%22farmer%27s%20organization%22&typeToAdd=journal_article&startIndexSearch=0
search.do?recordID=US201301615788
Fetch http://agris.fao.org/agris-search/search.do?recordID=US201301615788 to ../tmp/AGRIS_team_2_check\US201301615788.html
sleep 1s
search.do?recordID=US201900158325
Fetch http://agris.fao.org/agris-search/search.do?recordID=US201900158325 to ../tmp/AGRIS_team_2_check\US201900158325.html
sleep 1s
search.do?recordID=VN2008000147
Fetch http://agris.fao.org/agris-search/search.do?recordID=VN2008000147 to ../tmp/AGRIS_team_2_check\VN2008000147.html
sleep 1s
search.do?recordID=BD19900092252
Fetch http://agris.fao.org/agris-search/search.do?recordID=BD19900092252 to ../tmp/AGRIS_team_2_check\BD19900092252.html
sleep 1s
search.do?recordID=JP2006005781
Fetch http://agris.fao.org/agris-search/search.do?recordID=JP2006005781 to ../tmp/AGRIS_team_2_check\JP2006005781.html
sleep 1s
search.do?recordID=PH19960118094
Fet

sleep 1s
search.do?recordID=PH2009000380
Fetch http://agris.fao.org/agris-search/search.do?recordID=PH2009000380 to ../tmp/AGRIS_team_2_check\PH2009000380.html
sleep 1s
search.do?recordID=PH2008C00097
Fetch http://agris.fao.org/agris-search/search.do?recordID=PH2008C00097 to ../tmp/AGRIS_team_2_check\PH2008C00097.html
sleep 1s
search.do?recordID=JP2008006359
Fetch http://agris.fao.org/agris-search/search.do?recordID=JP2008006359 to ../tmp/AGRIS_team_2_check\JP2008006359.html
sleep 1s
sleep 3s


Page 5
Fetch http://agris.fao.org/agris-search/biblio.action?advQuery=%22farmer%27s%20organization%22&typeToAdd=journal_article&startIndexSearch=50
search.do?recordID=JP2006006452
Fetch http://agris.fao.org/agris-search/search.do?recordID=JP2006006452 to ../tmp/AGRIS_team_2_check\JP2006006452.html
sleep 1s
search.do?recordID=PH2005000520
Fetch http://agris.fao.org/agris-search/search.do?recordID=PH2005000520 to ../tmp/AGRIS_team_2_check\PH2005000520.html
sleep 1s
search.do?recordID=ID2002000088


sleep 1s
search.do?recordID=GB2012112740
Fetch http://agris.fao.org/agris-search/search.do?recordID=GB2012112740 to ../tmp/AGRIS_team_2_check\GB2012112740.html
sleep 1s
search.do?recordID=US2012203035
Fetch http://agris.fao.org/agris-search/search.do?recordID=US2012203035 to ../tmp/AGRIS_team_2_check\US2012203035.html
sleep 1s
search.do?recordID=US8527022
Fetch http://agris.fao.org/agris-search/search.do?recordID=US8527022 to ../tmp/AGRIS_team_2_check\US8527022.html
sleep 1s
search.do?recordID=QY2016600734
Fetch http://agris.fao.org/agris-search/search.do?recordID=QY2016600734 to ../tmp/AGRIS_team_2_check\QY2016600734.html
sleep 1s
search.do?recordID=CO19770185711
Fetch http://agris.fao.org/agris-search/search.do?recordID=CO19770185711 to ../tmp/AGRIS_team_2_check\CO19770185711.html
sleep 1s
search.do?recordID=AG19810651807
Fetch http://agris.fao.org/agris-search/search.do?recordID=AG19810651807 to ../tmp/AGRIS_team_2_check\AG19810651807.html
sleep 1s
search.do?recordID=US19780275790
F

In [14]:
_webcrawler_unep.prepare_dataset("../tmp/AGRIS_team_2_check", "../tmp/res_team_2.xlsx")

Saving...
Saved to ../tmp/res_team_2.xlsx


# IFPRI

In [387]:
doc = parse(fetch(
    r'http://ebrary.ifpri.org/cdm/search/collection/p15738coll9!p15738coll16!p15738coll17!p15738coll5!p15738coll2!p15738coll11!p15738coll10/searchterm/Discussion%20paper!smallholder*!Discussion%20paper!smallholder*!20080101-20181004/field/all!all!all!all!date/mode/exact!any!exact!any!exact/conn/and!and!and!and!and/page/1'
))

Fetch http://ebrary.ifpri.org/cdm/search/collection/p15738coll9!p15738coll16!p15738coll17!p15738coll5!p15738coll2!p15738coll11!p15738coll10/searchterm/Discussion%20paper!smallholder*!Discussion%20paper!smallholder*!20080101-20181004/field/all!all!all!all!date/mode/exact!any!exact!any!exact/conn/and!and!and!and!and/page/1


In [388]:
for el in doc.select('li.listItem > ul > li > div.marginTopTextAdjuster > a'):
    print(el.text)
    print('http://ebrary.ifpri.org' + el['href'])

Detailed crop suitability maps and an agricultural zonation scheme for Malawi: Map files
http://ebrary.ifpri.org/cdm/compoundobject/collection/p15738coll2/id/130561/rec/1
The future of small farms
http://ebrary.ifpri.org/cdm/compoundobject/collection/p15738coll2/id/64517/rec/2
Local markets, local varieties
http://ebrary.ifpri.org/cdm/compoundobject/collection/p15738coll2/id/24579/rec/3
Rich consumers and poor producers
http://ebrary.ifpri.org/cdm/compoundobject/collection/p15738coll2/id/28444/rec/4
The impact of the International Food Policy Research Institute's research program on rural finance policies for food security for the poor
http://ebrary.ifpri.org/cdm/compoundobject/collection/p15738coll2/id/81052/rec/5
New risks and opportunities for food security
http://ebrary.ifpri.org/cdm/compoundobject/collection/p15738coll2/id/60316/rec/6
Evaluating the impact of social networks in rural innovation systems
http://ebrary.ifpri.org/cdm/compoundobject/collection/p15738coll2/id/22304/rec/

In [60]:
import re
from webscrapping import webcrawler_ifpri
from webscrapping import webcrawler_base
webcrawler_ifpri = reload(webcrawler_ifpri)
webcrawler_base = reload(webcrawler_base)

_webcrawler_unep = webcrawler_ifpri.WebCrawlerIFPRI()
_webcrawler_unep.crawl_query('http://ebrary.ifpri.org/cdm/search/collection/p15738coll9!p15738coll16!p15738coll17!p15738coll5!p15738coll2!p15738coll11!p15738coll10/searchterm/Discussion%20paper!smallholder*!Discussion%20paper!smallholder*!20080101-20181004/field/all!all!all!all!date/mode/exact!any!exact!any!exact/conn/and!and!and!and!and/page/1', "../tmp/IFPRI")

In [61]:
_webcrawler_unep.prepare_dataset("../tmp/IFPRI", "../tmp/Datasets/IFPRI.xlsx")

# World Bank

In [18]:
doc = parse(fetch(
    r'https://openknowledge.worldbank.org/discover?query=smallholder&filtertype=dateIssued&filter_relational_operator=equals&filter=%5B2010+TO+2018%5D&page=1'
))

Fetch https://openknowledge.worldbank.org/discover?query=smallholder&filtertype=dateIssued&filter_relational_operator=equals&filter=%5B2010+TO+2018%5D&page=1


In [19]:
[
    'https://openknowledge.worldbank.org' + item['href'] + '?show=full'
    for item in doc.select('div > h4 > a')
]

['https://openknowledge.worldbank.org/handle/10986/29764?show=full',
 'https://openknowledge.worldbank.org/handle/10986/25786?show=full',
 'https://openknowledge.worldbank.org/handle/10986/30109?show=full',
 'https://openknowledge.worldbank.org/handle/10986/29543?show=full',
 'https://openknowledge.worldbank.org/handle/10986/12770?show=full',
 'https://openknowledge.worldbank.org/handle/10986/12613?show=full',
 'https://openknowledge.worldbank.org/handle/10986/25368?show=full',
 'https://openknowledge.worldbank.org/handle/10986/23498?show=full',
 'https://openknowledge.worldbank.org/handle/10986/29309?show=full',
 'https://openknowledge.worldbank.org/handle/10986/22158?show=full']

In [72]:
import re
from webscrapping import webcrawler_world_bank
from webscrapping import webcrawler_base
webcrawler_world_bank = reload(webcrawler_world_bank)

_webcrawler_unep = webcrawler_world_bank.WebCrawlerWorldBank()
_webcrawler_unep.crawl_query('https://openknowledge.worldbank.org/discover?query=smallholder&filtertype=dateIssued&filter_relational_operator=equals&filter=%5B2010+TO+2018%5D&page=1', "../tmp/WorldBank")

In [73]:
_webcrawler_unep.prepare_dataset("../tmp/WorldBank", "../tmp/Datasets/Worldbank.xlsx")

# GARDIAN

In [62]:
doc = parse(fetch(
    r'http://gardian.bigdata.cgiar.org/search.php#!/smallholder/all/and/%7B%22pub_sort%22:%22relevance%22,%22data_sort%22:%22relevance%22,%22pub_field%22:%22All%20fields%22%7D'
))

Fetch http://gardian.bigdata.cgiar.org/search.php#!/smallholder/all/and/%7B%22pub_sort%22:%22relevance%22,%22data_sort%22:%22relevance%22,%22pub_field%22:%22All%20fields%22%7D


In [65]:
jsn = requests.post('http://gardian.bigdata.cgiar.org/php_elastic/search_publication_advanced.php', 
               data={
                   'keywords': 'smallholder',
                   'from': 1,
                   'facet': 1,
                   'country': 'none',
                   'soption': 'and',
                   'year': 'none',
                   'center': 'none',
                   'field': '_all',
                   'type': 'none',
                   'sort': 'relevance'
               })

In [104]:
print(len(jsn['publications']['hits']['hits']))
jsn['publications']['hits']['hits']

10


[{'_index': 'publications_2_0',
  '_type': 'publications_2_0_records',
  '_id': 'AWLQyM8teko6lT8IQyho',
  '_score': 7.178628,
  '_source': {'cg.contributor': ['World Agroforestry Centre (ICRAF)'],
   'cg.identifier.url': ['http://www.worldagroforestry.org/publication/imperata-management-smallholder-extensionists-guide-rational-imperata-smallholder'],
   'cg.identifier.hdl': [''],
   'cg.identifier.doi': '',
   'cg.rights': 'restricted',
   'cg.identifier.citation': "S Rahayu, 'Imperata management for smallholder: an extensionist's guide to rational Imperata for smallholder', p.56, 1996",
   'cg.title': "Imperata management for smallholder: an extensionist's guide to rational Imperata for smallholder",
   'cg.description': '',
   'cg.date.production': '1996',
   'cg.date.distribution': '',
   'cg.language': '',
   'cg.type': 'Book / Monograph',
   'cg.creator': 'S Rahayu',
   'publication_id': 14708,
   'cg.subject.topics': [],
   'cg.country': [],
   'cg.subject.agrovoc': [{'Frequency'

In [57]:
query = "https://gardian.bigdata.cgiar.org/search.php#!/smallholder/all/and/%7B%22pub_sort%22:%22relevance%22,%22data_sort%22:%22relevance%22%7D/1/1"

In [58]:
doc = parse(fetch(query))

Fetch https://gardian.bigdata.cgiar.org/search.php#!/smallholder/all/and/%7B%22pub_sort%22:%22relevance%22,%22data_sort%22:%22relevance%22%7D/1/1


In [61]:
doc

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>GARDIAN Search</title>
<link href="https://fonts.googleapis.com/css?family=Roboto+Condensed:300,300i,400,400i,700,700i&amp;subset=latin-ext" rel="stylesheet"/>
<link href="assets/css/style2.css" rel="stylesheet"/>
<link href="assets/css/fontawesome-all.css" rel="stylesheet"/>
<link href="assets/css/simplebar.css" rel="stylesheet"/>
<script src="assets/js/scripts.build.js"></script>
<script src="assets/js/map.js"></script>
<script src="assets/js/simplebar.js"></script>
<script src="gardianGeotrellis/geotrellis.js"></script>
<script src="assets/bower_components/openlayers/ol.js"></script>
<script src="assets/bower_components/angular/angular.min.js"></script>
<script src="assets/bower_components/angular-sanitize/angular-sanitize.min.js"></script>
<script src="assets/bower_components/underscore/u

In [31]:
import re
from webscrapping import webcrawler_gardian_json
from webscrapping import webcrawler_base

_webcrawler_unep = webcrawler_gardian_json.WebCrawlerGARDIAN_JSON()
_webcrawler_unep.crawl_query('smallholder', "../tmp/GARDIAN_JSON")

In [153]:
_webcrawler_unep.prepare_dataset("../tmp/GARDIAN_JSON", "../tmp/Datasets/GARDIAN_JSON.xlsx")

# CEE - Collaboration for Environmental Evidence

http://www.environmentalevidence.org/completed-reviews?Terms=&Author=&Min_year=2008&Max_year=2018&Search=true

In [377]:
doc = parse(fetch(
    'http://www.environmentalevidence.org/completed-reviews?Terms=&Author=&Min_year=2008&Max_year=2018&Search=true'
))

Fetch http://www.environmentalevidence.org/completed-reviews?Terms=&Author=&Min_year=2008&Max_year=2018&Search=true


In [378]:
links = [
    item['href']
    for item in doc.select('div.crev-box > h2 > a')
]
print(len(links))
links

10


['http://www.environmentalevidence.org/completed-reviews/how-does-roadside-vegetation-management-affect-the-diversity-of-vascular-plants-and-invertebrates-a-systematic-review',
 'http://www.environmentalevidence.org/completed-reviews/what-specific-plant-traits-support-ecosystem-services-such-as-pollination-bio-control-and-water-quality-protection-in-temperate-climates-a-systematic-map-protocol',
 'http://www.environmentalevidence.org/completed-reviews/what-is-the-effect-of-prescribed-burning-in-temperate-and-boreal-forest-on-biodiversity-beyond-pyrophilous-and-saproxylic-species-a-systematic-review',
 'http://www.environmentalevidence.org/completed-reviews/what-are-the-impacts-of-manipulating-grazing-and-browsing-by-ungulates-on-plants-and-invertebrates-in-temperate-and-boreal-forests-a-systematic-review-protocol',
 'http://www.environmentalevidence.org/completed-reviews/the-multifunctional-roles-of-vegetated-strips-around-and-within-agricultural-fields-a-systematic-map-protocol',
 'ht

In [89]:
import re
from webscrapping import webcrawler_cee
from webscrapping import webcrawler_base
webcrawler_base = reload(webcrawler_base)
webcrawler_cee = reload(webcrawler_cee)

_webcrawler_unep = webcrawler_cee.WebCrawlerCEE()
_webcrawler_unep.crawl_query(f'http://www.environmentalevidence.org/completed-reviews/page/1?Terms&Author&Min_year=2008&Max_year=2018&Search=true#038;Author&Min_year=2008&Max_year=2018&Search=true', "../tmp/CEE")

In [90]:
_webcrawler_unep.prepare_dataset("../tmp/CEE", "../tmp/Datasets/CEE.xlsx")

# CAST - Council for Agricultural Science and Technology

http://www.cast-science.org/publications/

In [364]:
doc = parse(fetch(
    'http://www.cast-science.org/publications/'
))

Fetch http://www.cast-science.org/publications/


In [365]:
links = [item['href'] for item in doc.select('td.description h3 a')]
print(len(links))
links

10


['/publications/?reducing_the_impacts_of_agricultural_nutrients_on_water_quality_across_a_changing_landscape&show=product&productID=285025',
 '/publications/?reducing_the_impacts_of_agricultural_nutrients_on_water_quality_across_a_changing_landscape_qc&show=product&productID=285026',
 '/publications/?enabling_opensource_data_networks_in_public_agricultural_research&show=product&productID=285022',
 '/publications/?enabling_opensource_data_networks_in_public_agricultural_research_qc&show=product&productID=285023',
 '/publications/?aquifer_depletion_and_potential_impacts_on_longterm_irrigated_agricultural_productivity&show=product&productID=285019',
 '/publications/?aquifer_depletion_and_potential_impacts_on_longterm_irrigated_agricultural_productivity_qc&show=product&productID=285020',
 '/publications/?food_loss_and_waste&show=product&productID=284997',
 '/publications/?food_loss_and_waste_qc&show=product&productID=284998',
 '/publications/?impact_of_freerange_poultry_production_systems_

In [92]:
import re
from webscrapping import webcrawler_cast
from webscrapping import webcrawler_base
webcrawler_cast = reload(webcrawler_cast)

_webcrawler_unep = webcrawler_cast.WebCrawlerCAST()
_webcrawler_unep.crawl_query(f'http://www.cast-science.org/publications/index.cfm?view=list&page=1', "../tmp/CAST")

In [93]:
_webcrawler_unep.prepare_dataset("../tmp/CAST", "../tmp/Datasets/CAST.xlsx")

# IPA Innovation for Poverty Action (JPAL/ATAI impact evaluations)

https://www.poverty-action.org/program-area/agriculture/publications

In [24]:
doc = parse(fetch(
    'https://www.poverty-action.org/program-area/agriculture/publications'
))

Fetch https://www.poverty-action.org/program-area/agriculture/publications


In [25]:
links = [item['href'] for item in doc.select('span.field-content a')]
print(len(links))
links

15


['/publication/rwanda-country-brief',
 '/publication/pass-through-competition-and-entry-agricultural-markets-experimental-evidence-kenya',
 '/publication/self-selection-payments-ecosystem-services-programs',
 '/publication/comparing-yields-and-profits-seed-varieties-northern-ghana',
 '/publication/disseminating-innovative-resources-and-technologies-smallholder-farmers-ghana-results-0',
 '/publication/testing-agricultural-technologies-preliminary-results',
 '/publication/disseminating-innovative-resources-and-technologies-smallholder-farmers-ghana-results',
 '/publication/sell-low-and-buy-high-arbitrage-and-local-price-effects-kenyan-markets',
 '/publication/myanmar-country-brief',
 '/publication/agriculture-program-area-brief',
 '/publication/diffusion-agricultural-information-within-social-networks-evidence-gender-inequalities',
 '/publication/zambia-bulletin-volume-9-10',
 '/publication/selling-low-and-buying-high-arbitrage-puzzle-kenyan-villages',
 '/publication/sierra-leone-country

In [95]:
import re
from webscrapping import webcrawler_ipa
from webscrapping import webcrawler_base
webcrawler_ipa = reload(webcrawler_ipa)

_webcrawler_unep = webcrawler_ipa.WebCrawlerIPA()
_webcrawler_unep.crawl_query(f'https://www.poverty-action.org/program-area/agriculture/publications?page=0', "../tmp/IPA")

In [96]:
_webcrawler_unep.prepare_dataset("../tmp/IPA", "../tmp/Datasets/IPA.xlsx")

# ODI - Overseas Development Institute (ODI)

https://www.odi.org/publications

In [22]:
doc = parse(fetch(
    'https://www.odi.org/publications?page=3'
))

Fetch https://www.odi.org/publications?page=3


In [32]:
links = [item['href'] for item in doc.select('h3.node__title a')]
print(len(links))
links

5


['https://www.odi.org/publications/11234-thinking-and-working-political-settlements-case-tanzania',
 'https://www.odi.org/publications/11233-migration-policy-practice-83-september-october-2018',
 'https://www.odi.org/publications/11241-donor-proliferation-what-ends-new-donor-countries-and-search-legitimacy',
 'https://www.odi.org/publications/11228-between-work-and-care-older-womens-economic-empowerment',
 'https://www.odi.org/publications/11230-brown-green-g20-transition-towards-low-carbon-economy']

In [20]:
import re
from webscrapping import webcrawler_odi
from webscrapping import webcrawler_base
webcrawler_odi = reload(webcrawler_odi)

_webcrawler_unep = webcrawler_odi.WebCrawlerODI()
data_from = '2008-01-01'
to_date = '2019-01-01'
#_webcrawler_unep.crawl_query(f'https://www.odi.org/publications?page=0&f%5B0%5D=dm_field_start_date%3A%5B{data_from}T00%3A00%3A00Z%20TO%20{to_date}T00%3A00%3A00Z%5D', "../tmp/ODI")
_webcrawler_unep.crawl_query("https://www.odi.org/search/site/Smallholder%20AND%20modern%20markets","../tmp/odi_try")



Page 0
Fetch https://www.odi.org/search/site/Smallholder%20AND%20modern%20markets?page=0&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
https://www.odi.org/publications/4226-farmer-federation-coffee-colombia
Fetch https://www.odi.org/publications/4226-farmer-federation-coffee-colombia to ../tmp/odi_try\4226-farmer-federation-coffee-colombia.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review
Fetch https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review to ../tmp/odi_try\1656-theory-practice-plantation-agriculture-economic-review.html
sleep 1s
https://www.odi.org/publications/4253-viet-nam-farmers-livestock-schools
Fetch https://www.odi.org/publications/4253-viet-nam-farmers-livestock-schools to ../tmp/odi_try\4253-viet-nam-farmers-livestock-schools.html
sleep 1s
https://www.odi.org/publications/2050-reaching-poor-developing-poverty-screen-agricultura

In [100]:
_webcrawler_unep.prepare_dataset("../tmp/ODI","../tmp/Datasets/ODI.xlsx")

# DFID - UK Department for International Development

https://www.gov.uk/dfid-research-outputs?first_published_at%5Bfrom%5D=2008&first_published_at%5Bto%5D=&keywords=smallholder&page=1

In [357]:
doc = parse(fetch(
    'https://www.gov.uk/dfid-research-outputs?first_published_at%5Bfrom%5D=2008&first_published_at%5Bto%5D=&keywords=smallholder&page=1'
))

Fetch https://www.gov.uk/dfid-research-outputs?first_published_at%5Bfrom%5D=2008&first_published_at%5Bto%5D=&keywords=smallholder&page=1


In [363]:
links = [item['href'] for item in doc.select('li.document a')]
print(len(links))

50


In [106]:
import re
from webscrapping import webcrawler_dfid
from webscrapping import webcrawler_base
webcrawler_dfid = reload(webcrawler_dfid)

_webcrawler_unep = webcrawler_dfid.WebCrawlerDFID()
_webcrawler_unep.crawl_query('https://www.gov.uk/dfid-research-outputs?first_published_at%5Bfrom%5D=2008&first_published_at%5Bto%5D=&keywords=smallholder&page=1', "../tmp/DFID")

In [107]:
_webcrawler_unep.prepare_dataset("../tmp/DFID","../tmp/Datasets/DFID.xlsx")

# IFAD - International Fund for Agricultural Development

https://www.ifad.org/en/web/knowledge/research?mode=search&page=1

In [336]:
doc = parse(fetch(
    'https://www.ifad.org/en/web/knowledge/research?mode=search&page=1', custom_headers=IFAD_HEADERS
))

Fetch https://www.ifad.org/en/web/knowledge/research?mode=search&page=1


In [330]:
links = [item['href'] for item 
         in doc.select('div.abstract-row div.abstract-row-content-wrapper a')
         if item['href'].startswith('/web')]
print(len(links))
links

15


['/web/knowledge/publication/asset/41123411',
 '/web/knowledge/publication/asset/41116368',
 '/web/knowledge/publication/asset/41094963',
 '/web/knowledge/publication/asset/41115553',
 '/web/knowledge/publication/asset/41082658',
 '/web/knowledge/publication/asset/41067095',
 '/web/knowledge/publication/asset/40974109',
 '/web/knowledge/publication/asset/40956425',
 '/web/knowledge/publication/asset/41117753',
 '/web/knowledge/publication/asset/40952390',
 '/web/knowledge/publication/asset/41096296',
 '/web/knowledge/publication/asset/41096501',
 '/web/knowledge/publication/asset/41097486',
 '/web/knowledge/publication/asset/41097252',
 '/web/knowledge/publication/asset/41096980']

In [108]:
import re
from webscrapping import webcrawler_ifad
from webscrapping import webcrawler_base
webcrawler_base = reload(webcrawler_base)
webcrawler_ifad = reload(webcrawler_ifad)

_webcrawler_unep = webcrawler_ifad.WebCrawlerIFAD()
_webcrawler_unep.crawl_query('https://www.ifad.org/en/web/knowledge/research?mode=search&page=1', "../tmp/IFAD")

In [109]:
_webcrawler_unep.prepare_dataset("../tmp/IFAD","../tmp/Datasets/IFAD.xlsx")

# IIED - International Institute for Environment and Development

https://www.ifad.org/en/web/knowledge/research?mode=search&page=1

In [54]:
doc = parse(fetch(
    'http://pubs.iied.org/search/?k=smallholder&p=1'
))
links = [item['href'] for item 
         in doc.select('li a')]
print(len(links))
links

Fetch http://pubs.iied.org/search/?k=smallholder&p=1
10


['/13606IIED/?k=smallholder',
 '/17618IIED/?k=smallholder',
 '/G04311/?k=smallholder',
 '/G04271/?k=smallholder',
 '/G04288/?k=smallholder',
 '/13595IIED/?k=smallholder',
 '/17447IIED/?k=smallholder',
 '/17610IIED/?k=smallholder',
 '/17610FIIED/?k=smallholder',
 '/G04261/?k=smallholder']

In [111]:
import re
from webscrapping import webcrawler_iied
from webscrapping import webcrawler_base
webcrawler_iied = reload(webcrawler_iied)

_webcrawler_unep = webcrawler_iied.WebCrawlerIIED()
_webcrawler_unep.crawl_query('http://pubs.iied.org/search/?k=smallholder&p=1', "../tmp/IIED")

In [112]:
_webcrawler_unep.prepare_dataset("../tmp/IIED","../tmp/Datasets/IIED.xlsx")

# CSIRO https://publications.csiro.au/rpr/home 

In [321]:
query = "https://publications.csiro.au/rpr/search?q=smallholder%20farming&p=1&rpp=25&sb=RECENT&expert=false&soud=false&dr=all&list=SEA"

In [324]:
doc = parse(fetch(query))

In [23]:
links = [item['href'] for item 
         in doc.select('div.searchResultSection a')]
print(len(links))

23


In [118]:
import re
from webscrapping import webcrawler_csiro
from webscrapping import webcrawler_base
webcrawler_csiro = reload(webcrawler_csiro)

_webcrawler_unep = webcrawler_csiro.WebCrawlerCSIRO()
_webcrawler_unep.crawl_query(query, "../tmp/CSIRO")

In [119]:
_webcrawler_unep.prepare_dataset("../tmp/CSIRO","../tmp/Datasets/CSIRO.xlsx")

# French Agricultural Research Centre for International Development (CIRAD)

In [308]:
query = "https://agritrop.cirad.fr/cgi/search/simple?q=smallholder&_action_search=Search&_order=bytitle&basic_srchtype=ALL&_satisfyall=ALL&search_offset=0"

In [309]:
doc = parse(fetch(query))

Fetch https://agritrop.cirad.fr/cgi/search/simple?q=smallholder&_action_search=Search&_order=bytitle&basic_srchtype=ALL&_satisfyall=ALL&search_offset=0


In [312]:
links = [item['href'] for item 
         in doc.select('tr.ep_search_result a') if item["href"].startswith("http://agritrop.cirad.fr")]
print(len(links))

21


In [124]:
import re
from webscrapping import webcrawler_cirad
from webscrapping import webcrawler_base
webcrawler_cirad = reload(webcrawler_cirad)

_webcrawler_unep = webcrawler_cirad.WebCrawlerCIRAD()
_webcrawler_unep.crawl_query(query, "../tmp/CIRAD", 0)

In [125]:
_webcrawler_unep.prepare_dataset("../tmp/CIRAD","../tmp/Datasets/CIRAD.xlsx")

In [76]:
import re
from webscrapping import webcrawler_cirad_fr
from webscrapping import webcrawler_base
webcrawler_cirad_fr = reload(webcrawler_cirad_fr)

_webcrawler_unep = webcrawler_cirad_fr.WebCrawlerCIRAD_fr()
_webcrawler_unep.crawl_query("https://www.cirad.fr/en/content/search?SubTreeArray%5B%5D=2&SearchText=environment+AND+incentive+AND+farmer&submit.x=0&submit.y=0", "../tmp/CIRAD_fr", 0)



Page 0
Fetch https://www.cirad.fr/en/content/search?SubTreeArray%5B%5D=2&SearchText=environment+AND+incentive+AND+farmer&submit.x=0&submit.y=0/(offset)/0
/en/news/all-news-items/articles/2019/science/environmental-policies-payments-may-be-counter-productive
Fetch https://www.cirad.fr/en/news/all-news-items/articles/2019/science/environmental-policies-payments-may-be-counter-productive to ../tmp/CIRAD_fr\environmental-policies-payments-may-be-counter-productive.html
Already exists. Skipped.
sleep 1s
/en/our-research/research-results/2009/private-goods-and-public-services-the-need-for-compromise
Fetch https://www.cirad.fr/en/our-research/research-results/2009/private-goods-and-public-services-the-need-for-compromise to ../tmp/CIRAD_fr\private-goods-and-public-services-the-need-for-compromise.html
Already exists. Skipped.
sleep 1s
/en/our-research/research-results/2015/urban-market-gardening-in-vietnam-sanitary-quality-means-providing-producers-groups-with-technical-advice
Fetch https:/

In [77]:
_webcrawler_unep.prepare_dataset("../tmp/CIRAD_fr","../tmp/Datasets/CIRAD_fr.xlsx")

# AgEcon Search

In [15]:
query="https://ageconsearch.umn.edu/search?ln=en&p=farmer+environment+incentives+adoption&jrec=41"

In [21]:
doc = parse(fetch(query))
links = [item['href'].split("files/")[0] for item 
         in doc.select('div.result-row a') if item['href'].startswith("/record/")]
print(len(links))
links

Fetch https://ageconsearch.umn.edu/search?ln=en&p=farmer+environment+incentives+adoption&jrec=41
0


[]

In [129]:
import re
from webscrapping import webcrawler_agecon
from webscrapping import webcrawler_base
webcrawler_agecon = reload(webcrawler_agecon)

_webcrawler_unep = webcrawler_agecon.WebCrawlerAGECON()
_webcrawler_unep.crawl_query(query, "../tmp/AgEcon", 0)

In [130]:
_webcrawler_unep.prepare_dataset("../tmp/AgEcon","../tmp/Datasets/AgEcon.xlsx")

# EMBRAPA

In [23]:
query="https://www.embrapa.br/busca-de-publicacoes/-/publicacao/busca/smallholder+farming?_buscapublicacao_WAR_pcebusca6_1portlet_delta=10&_buscapublicacao_WAR_pcebusca6_1portlet_keywords=&_buscapublicacao_WAR_pcebusca6_1portlet_advancedSearch=false&_buscapublicacao_WAR_pcebusca6_1portlet_andOperator=true&_buscapublicacao_WAR_pcebusca6_1portlet_resetCur=false&_buscapublicacao_WAR_pcebusca6_1portlet_cur=2"

In [4]:
doc = parse(fetch(query))

Fetch https://www.embrapa.br/en/busca-geral/-/busca/environment+incentive+farmer?_buscageral_WAR_pcebusca6_1portlet_keywords=&_buscageral_WAR_pcebusca6_1portlet_advancedSearch=false&_buscageral_WAR_pcebusca6_1portlet_delta=5&_buscageral_WAR_pcebusca6_1portlet_resetCur=false&_buscageral_WAR_pcebusca6_1portlet_andOperator=true&_buscageral_WAR_pcebusca6_1portlet_cur=2


In [182]:
doc = parse(fetch(query))
links = [item['href'] for item 
         in doc.select('div.titulo a')]
print(len(links))
links

Fetch https://www.embrapa.br/busca-de-publicacoes/-/publicacao/busca/smallholder+farming?_buscapublicacao_WAR_pcebusca6_1portlet_delta=10&_buscapublicacao_WAR_pcebusca6_1portlet_keywords=&_buscapublicacao_WAR_pcebusca6_1portlet_advancedSearch=false&_buscapublicacao_WAR_pcebusca6_1portlet_andOperator=true&_buscapublicacao_WAR_pcebusca6_1portlet_resetCur=false&_buscapublicacao_WAR_pcebusca6_1portlet_cur=2
10


['/busca-de-publicacoes/-/publicacao/1087829/fitting-brazilian-livestock-production-to-changes-in-natural-and-political-environments',
 '/busca-de-publicacoes/-/publicacao/965693/praticas-agroecologicas-de-convivencia-com-semiarido-adotadas-por-agricultores-familiares-no-sertao-cearense',
 '/busca-de-publicacoes/-/publicacao/133084/farmers-organizations-and-acess-to-innovations-pathways-of-socioeconomic-change-in-massaroca-bahia-state-brazil',
 '/busca-de-publicacoes/-/publicacao/1004277/caracterizacao-dos-sistemas-de-producao-de-leite-caprino-no-estado-do-ceara-brasil',
 '/busca-de-publicacoes/-/publicacao/661839/evaluation-of-sustainability-in-eastern-amazon-under-proambiente-program',
 '/busca-de-publicacoes/-/publicacao/15698/the-balance-between-sustainability-dimensions-ecological-base-familiar-smallholders-case-from-coagrosolbrazil-',
 '/busca-de-publicacoes/-/publicacao/1088292/projeto-sustentare-trabalhando-para-e-com-os-agricultores-do-municipio-de-sobral-ceara-em-busca-do-des

In [34]:
q = "https://www.embrapa.br/en/busca-de-publicacoes/-/publicacao/busca/hhjhkhj"
if re.search(r"https://www.embrapa.br/en/busca-de-publicacoes/-/publicacao/busca/.+\??.*", q) != None:
    print("sd")

sd


In [29]:
import re
from webscrapping import webcrawler_embrapa
from webscrapping import webcrawler_base
webcrawler_embrapa = reload(webcrawler_embrapa)

_webcrawler_unep = webcrawler_embrapa.WebCrawlerEMBRAPA()
_webcrawler_unep.crawl_query("https://www.embrapa.br/en/busca-de-publicacoes/-/publicacao/busca/smallholder+farming?p_auth=pEVc36Iv&p_p_id=buscageral_WAR_pcebusca6_1portlet&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_buscageral_WAR_pcebusca6_1portlet_javax.portlet.action=buscar&_buscageral_WAR_pcebusca6_1portlet_delta=10", "../tmp/EMBRAPA_1")



Page 1
Fetch https://www.embrapa.br/en/busca-de-publicacoes/-/publicacao/busca/smallholder+farming?p_auth=pEVc36Iv&p_p_id=buscageral_WAR_pcebusca6_1portlet&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_buscageral_WAR_pcebusca6_1portlet_javax.portlet.action=buscar&_buscageral_WAR_pcebusca6_1portlet_delta=10&portlet_cur=1
/busca-de-publicacoes/-/publicacao/1037679/feasibility-and-competitiveness-of-intensive-smallholder-dairy-farming-in-brazil-in-comparison-with-soya-and-sugarcane-case-study-of-the-balde-cheio-programme
Fetch https://www.embrapa.br/busca-de-publicacoes/-/publicacao/1037679/feasibility-and-competitiveness-of-intensive-smallholder-dairy-farming-in-brazil-in-comparison-with-soya-and-sugarcane-case-study-of-the-balde-cheio-programme to ../tmp/EMBRAPA_1\1037679.html
Already exists. Skipped.
sleep 1s
/busca-de-publicacoes/-/publicacao/1081461/agronomic-performance-of-brs-itaim-cowpea-beans-at-different-planting-densities-under-no-tillage

KeyboardInterrupt: 

In [133]:
_webcrawler_unep.prepare_dataset("../tmp/EMBRAPA","../tmp/Datasets/EMBRAPA.xlsx")

# WHO

In [295]:
query = "https://apps.who.int/iris/discover?rpp=10&etal=0&query=smallholder&scope=/&group_by=none&page=1"

In [215]:
doc = parse(fetch(query))
links = [item['href'] for item 
         in doc.select('.artifact-title a')]
print(len(links))
links

Fetch https://apps.who.int/iris/discover?rpp=10&etal=0&query=smallholder&scope=/&group_by=none&page=1
10


['/iris/handle/10665/280104',
 '/iris/handle/10665/311643',
 '/iris/handle/10665/311711',
 '/iris/handle/10665/277440',
 '/iris/handle/10665/274739',
 '/iris/handle/10665/278948',
 '/iris/handle/10665/260525',
 '/iris/handle/10665/273038',
 '/iris/handle/10665/311280',
 '/iris/handle/10665/275518']

In [135]:
import re
import sys

sys.path.append('../src')
from webscrapping import webcrawler_who
from webscrapping import webcrawler_base
webcrawler_who =reload(webcrawler_who)

_webcrawler_unep = webcrawler_who.WebCrawlerWHO()
_webcrawler_unep.crawl_query(query, "../tmp/WHO")

In [136]:
_webcrawler_unep.prepare_dataset("../tmp/WHO","../tmp/Datasets/WHO.xlsx")

# UNEP

In [444]:
query = "https://wedocs.unep.org/handle/20.500.11822/1/discover?rpp=10&etal=0&query=farming&group_by=none&page=1"

In [280]:
doc = parse(fetch(query))
links = [item['href'] for item 
         in doc.select('.artifact-description a')]
print(len(links))
links

Fetch https://wedocs.unep.org/handle/20.500.11822/1/discover?rpp=10&etal=0&query=farming&group_by=none&page=1
10


['/handle/20.500.11822/22930',
 '/handle/20.500.11822/22933',
 '/handle/20.500.11822/22931',
 '/handle/20.500.11822/22926',
 '/handle/20.500.11822/22928',
 '/handle/20.500.11822/22932',
 '/handle/20.500.11822/22929',
 '/handle/20.500.11822/22951',
 '/handle/20.500.11822/22936',
 '/handle/20.500.11822/22949']

In [144]:
import re
from webscrapping import webcrawler_unep
from webscrapping import webcrawler_base
from importlib import reload
webcrawler_unep = reload(webcrawler_unep)
webcrawler_base = reload(webcrawler_base)

_webcrawler_unep = webcrawler_unep.WebCrawlerUNEP()
_webcrawler_unep.crawl_query(query, "../tmp/UNEP")

In [145]:
_webcrawler_unep.prepare_dataset("../tmp/UNEP","../tmp/Datasets/UNEP.xlsx")

# WFP

In [343]:
query = "https://www1.wfp.org/publications?text=smallholder&page=0"

In [344]:
doc = parse(fetch(query))
links = [item['href'] for item 
         in doc.select('.db.lh-heading.fs4 a')]
print(len(links))

Fetch https://www1.wfp.org/publications?text=smallholder&page=0
20


['https://www1.wfp.org/publications/malawi-country-strategic-plan-strategic-objectives',
 'https://www1.wfp.org/publications/r4-rural-resilience-initiative-quarterly-report-april-june-2018',
 'https://www1.wfp.org/publications/zambia-country-programme-200891-mid-term-evaluation',
 'https://www1.wfp.org/publications/2018-using-insurance-protect-farmers-against-drought-senegal',
 'https://www1.wfp.org/publications/2017-smallholder-agriculture-market-sams-mali',
 'https://www1.wfp.org/publications/2017-wfp-zambia',
 'https://www1.wfp.org/publications/2017-weather-risk-management-facility-wrmf',
 'https://www1.wfp.org/publications/2017-remote-sensing-index-insurance-findings-and-lessons-learned-smallholder-agriculture',
 'https://www1.wfp.org/publications/2017-pro-smallholder-food-assistance-strategy-boosting-smallholder-resilience-and-market-acc',
 'https://www1.wfp.org/publications/publications-page-%E2%80%98r4-rural-resilience-initiative-quarterly-report-october-%E2%80%93-december-2016'

In [147]:
import re
from webscrapping import webcrawler_wfp
from webscrapping import webcrawler_base

_webcrawler_unep = webcrawler_wfp.WebCrawlerWFP()
_webcrawler_unep.crawl_query(query, "../tmp/WFP")

In [148]:
_webcrawler_unep.prepare_dataset("../tmp/WFP","../tmp/Datasets/WFP.xlsx")

# DATAD Africa Theses and Dissertations

In [460]:
query = "http://datad.aau.org/discover?rpp=10&etal=0&query=climate+AND+seed+OR+germplasm&scope=/&group_by=none&page=1"

In [461]:
doc = parse(fetch(query))
links = [item['href'] for item 
         in doc.select('.artifact-title a')]
print(len(links))
links

Fetch http://datad.aau.org/discover?rpp=10&etal=0&query=climate+AND+seed+OR+germplasm&scope=/&group_by=none&page=1
10


['/handle/123456789/11371',
 '/handle/123456789/8201',
 '/handle/11660/925',
 '/handle/123456789/31055',
 '/handle/123456789/11459',
 '/handle/123456789/12859',
 '/handle/11660/8118',
 '/handle/123456789/9236',
 '/handle/123456789/9237',
 '/handle/11660/1371']

In [150]:
import re
from webscrapping import webcrawler_datad
from webscrapping import webcrawler_base
webcrawler_datad = reload(webcrawler_datad)

_webcrawler_unep = webcrawler_datad.WebCrawlerDATAD()
_webcrawler_unep.crawl_query(query, "../tmp/DATAD")

In [151]:
_webcrawler_unep.prepare_dataset("../tmp/DATAD","../tmp/Datasets/DATAD.xlsx")

# Campbell collaboration

In [28]:
query = "https://www.campbellcollaboration.org/website-search?searchword=environment%20and%20incentive%20and%20farmer&start=0"

In [29]:
doc = parse(fetch(query))
links = [item['href'] for item 
         in doc.select('.result-title a')]
print(len(links))
links

Fetch https://www.campbellcollaboration.org/website-search?searchword=environment%20and%20incentive%20and%20farmer&start=0
1


['/library/contract-farming-and-smallholders-income.html']

In [43]:
import re
import sys
from importlib import reload
sys.path.append('../src')

from webscrapping import webcrawler_campbell
from webscrapping import webcrawler_base
webcrawler_campbell = reload(webcrawler_campbell)
webcrawler_base = reload(webcrawler_base)

_webcrawler_unep = webcrawler_campbell.WebCrawlerCampbell()
_webcrawler_unep.crawl_query(query, "../tmp/Campbell")

In [44]:
_webcrawler_unep.prepare_dataset("../tmp/Campbell","../tmp/Datasets/Campbell.xlsx")

# Main processing

In [1]:
import sys
from importlib import reload
sys.path.append('../src')
from webscrapping import webcrawler_main
webcrawler_main = reload(webcrawler_main)

for i in [3,4,5,6]:
    _webcrawler_main = webcrawler_main.WebcrawlerMain()
    _webcrawler_main.crawl_all_sources_from_file("team_%d.txt"%i,"../tmp/Team_%d"%i)
    _webcrawler_main.save_dataset_from_all_sources("../tmp/Team_%d"%i)

Started querying for source: https://www.odi.org


Page 0
Fetch https://www.odi.org/search/site/Smallholder%20AND%20modern%20markets?page=0&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
https://www.odi.org/publications/4226-farmer-federation-coffee-colombia
Fetch https://www.odi.org/publications/4226-farmer-federation-coffee-colombia to ../tmp/Team_3\www.odi.org\4226-farmer-federation-coffee-colombia.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review
Fetch https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review to ../tmp/Team_3\www.odi.org\1656-theory-practice-plantation-agriculture-economic-review.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4253-viet-nam-farmers-livestock-schools
Fetch https://www.odi.org/publications/4253-viet-nam-farmers-livestock-schools to ../tmp/Team_3\www.odi.org\4253-viet-nam-farmers-livestock-scho

https://www.odi.org/publications/7252-agricultural-research-extension-network-agren-newsletter-no.%2047
Fetch https://www.odi.org/publications/7252-agricultural-research-extension-network-agren-newsletter-no.%2047 to ../tmp/Team_3\www.odi.org\7252-agricultural-research-extension-network-agren-newsletter-no.%2047.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/2050-reaching-poor-developing-poverty-screen-agricultural-research-proposals
Fetch https://www.odi.org/publications/2050-reaching-poor-developing-poverty-screen-agricultural-research-proposals to ../tmp/Team_3\www.odi.org\2050-reaching-poor-developing-poverty-screen-agricultural-research-proposals.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4373-agren-newsletter-no-50
Fetch https://www.odi.org/publications/4373-agren-newsletter-no-50 to ../tmp/Team_3\www.odi.org\4373-agren-newsletter-no-50.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/1661-performance-se

https://www.odi.org/publications/3441-smallholder-dairy-project-bibliography
Fetch https://www.odi.org/publications/3441-smallholder-dairy-project-bibliography to ../tmp/Team_3\www.odi.org\3441-smallholder-dairy-project-bibliography.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4279-fair-trade-organic-andes-coffee
Fetch https://www.odi.org/publications/4279-fair-trade-organic-andes-coffee to ../tmp/Team_3\www.odi.org\4279-fair-trade-organic-andes-coffee.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4226-farmer-federation-coffee-colombia
Fetch https://www.odi.org/publications/4226-farmer-federation-coffee-colombia to ../tmp/Team_3\www.odi.org\4226-farmer-federation-coffee-colombia.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/10524-evaluative-learning-resilience-providing-humanitarian-assistance-sahel-emergencies-phase
Fetch https://www.odi.org/publications/10524-evaluative-learning-resilience-providing-humani

https://www.odi.org/publications/4704-sector-based-approaches-agriculture
Fetch https://www.odi.org/publications/4704-sector-based-approaches-agriculture to ../tmp/Team_3\www.odi.org\4704-sector-based-approaches-agriculture.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/2051-policy-research-and-african-agriculture-time-dose-reality
Fetch https://www.odi.org/publications/2051-policy-research-and-african-agriculture-time-dose-reality to ../tmp/Team_3\www.odi.org\2051-policy-research-and-african-agriculture-time-dose-reality.html
Already exists. Skipped.
sleep 1s
sleep 3s


Page 3
Fetch https://www.odi.org/search/site/Smallholder%20AND%20service%20provider?page=3&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
https://www.odi.org/publications/1777-strengthening-enabling-environment-agricultural-technology-development-sub-saharan-africa
Fetch https://www.odi.org/publications/1777-strengthening-enabling-environment-agricultural-technology-developme

https://www.odi.org/publications/7252-agricultural-research-extension-network-agren-newsletter-no.%2047
Fetch https://www.odi.org/publications/7252-agricultural-research-extension-network-agren-newsletter-no.%2047 to ../tmp/Team_3\www.odi.org\7252-agricultural-research-extension-network-agren-newsletter-no.%2047.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4818-rural-diversification-what-hope-poor
Fetch https://www.odi.org/publications/4818-rural-diversification-what-hope-poor to ../tmp/Team_3\www.odi.org\4818-rural-diversification-what-hope-poor.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/1661-performance-seed-sector-malawi-analysis-influence-organisational-structure
Fetch https://www.odi.org/publications/1661-performance-seed-sector-malawi-analysis-influence-organisational-structure to ../tmp/Team_3\www.odi.org\1661-performance-seed-sector-malawi-analysis-influence-organisational-structure.html
Already exists. Skipped.
sleep 1s
s

sleep 3s


Page 1
Fetch https://www.odi.org/search/site/adapt%2A%20AND%20smallholder%20AND%20germplasm?page=1&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
sleep 3s
Done!
Saving...
Saved to ../tmp/Team_4\Team_4.xlsx
Started querying for source: https://www.odi.org


Page 0
Fetch https://www.odi.org/search/site/Livestock%20AND%20feed*%20AND%20small*%20farmer?page=0&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
https://www.odi.org/publications/4253-viet-nam-farmers-livestock-schools
Fetch https://www.odi.org/publications/4253-viet-nam-farmers-livestock-schools to ../tmp/Team_5\www.odi.org\4253-viet-nam-farmers-livestock-schools.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4472-pastoralism-agricultural-extension-intensification-africa
Fetch https://www.odi.org/publications/4472-pastoralism-agricultural-extension-intensification-africa to ../tmp/Team_5\www.odi.org\4472-pastoralism-agricultural-extension-intensification-afri

https://www.odi.org/publications/4285-participation-approaches-sustainable-agriculture-malawi
Fetch https://www.odi.org/publications/4285-participation-approaches-sustainable-agriculture-malawi to ../tmp/Team_5\www.odi.org\4285-participation-approaches-sustainable-agriculture-malawi.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4386-income-diversity-sudan-cattle-pastoralism
Fetch https://www.odi.org/publications/4386-income-diversity-sudan-cattle-pastoralism to ../tmp/Team_5\www.odi.org\4386-income-diversity-sudan-cattle-pastoralism.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/7250-agricultural-research-extension-network-agren-newsletter-no.%2044
Fetch https://www.odi.org/publications/7250-agricultural-research-extension-network-agren-newsletter-no.%2044 to ../tmp/Team_5\www.odi.org\7250-agricultural-research-extension-network-agren-newsletter-no.%2044.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4461-polit

https://www.odi.org/publications/4224-soil-water-conservation-household-investment
Fetch https://www.odi.org/publications/4224-soil-water-conservation-household-investment to ../tmp/Team_6\www.odi.org\4224-soil-water-conservation-household-investment.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review
Fetch https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review to ../tmp/Team_6\www.odi.org\1656-theory-practice-plantation-agriculture-economic-review.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4280-forage-crops-sustainable-agriculture
Fetch https://www.odi.org/publications/4280-forage-crops-sustainable-agriculture to ../tmp/Team_6\www.odi.org\4280-forage-crops-sustainable-agriculture.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/10882-land-population-and-agricultural-investment-africa
Fetch https://www.odi.org/publicat

sleep 3s


Page 3
Fetch https://www.odi.org/search/site/smallholder%20AND%20water%20AND%20yield?page=3&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
https://www.odi.org/publications/1712-livestock-wildlife
Fetch https://www.odi.org/publications/1712-livestock-wildlife to ../tmp/Team_6\www.odi.org\1712-livestock-wildlife.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/5057-vietnam-viet-nam-economic-growth-poverty-development-progress
Fetch https://www.odi.org/publications/5057-vietnam-viet-nam-economic-growth-poverty-development-progress to ../tmp/Team_6\www.odi.org\5057-vietnam-viet-nam-economic-growth-poverty-development-progress.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/10884-2016-dfid-land-portfolio-overview
Fetch https://www.odi.org/publications/10884-2016-dfid-land-portfolio-overview to ../tmp/Team_6\www.odi.org\10884-2016-dfid-land-portfolio-overview.html
Already exists. Skipped.
sleep 1s
sleep 3s


Page 4




Page 3
Fetch https://www.odi.org/search/site/smallholder%20AND%20arid%20AND%20yield?page=3&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
sleep 3s
Done!
Started querying for source: https://www.odi.org


Page 0
Fetch https://www.odi.org/search/site/smallholder%20AND%20drought%20AND%20yield?page=0&f%5B0%5D=bundle%3Aresource&f%5B1%5D=sm_field_theme%3Anode%3A17399
https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review
Fetch https://www.odi.org/publications/1656-theory-practice-plantation-agriculture-economic-review to ../tmp/Team_6\www.odi.org\1656-theory-practice-plantation-agriculture-economic-review.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications/4227-ganyu-rural-labour-market-relations
Fetch https://www.odi.org/publications/4227-ganyu-rural-labour-market-relations to ../tmp/Team_6\www.odi.org\4227-ganyu-rural-labour-market-relations.html
Already exists. Skipped.
sleep 1s
https://www.odi.org/publications

In [ ]:
import sys
from importlib import reload
sys.path.append('../src')
from webscrapping import webcrawler_main
webcrawler_main = reload(webcrawler_main)

_webcrawler_main = webcrawler_main.WebcrawlerMain()
_webcrawler_main.crawl_all_sources_from_file("team_6_addition.txt","../tmp/team_6_addition")

Unknown source domain: #https://agritrop.cirad.fr
Unknown source domain: #https://agritrop.cirad.fr
Unknown source domain: #https://agritrop.cirad.fr
Unknown source domain: #https://agritrop.cirad.fr
Unknown source domain: #https://agritrop.cirad.fr
Started querying for source: https://agritrop.cirad.fr


Page 0
Fetch https://agritrop.cirad.fr/cgi/search/archive/advanced?screen=Search&dataset=archive&_action_search=Search&crd_mot_merge=ALL&crd_mot=small-scale+water&crd_mot_sujet_merge=ALL&crd_mot_sujet=&crd_titre_recherche_merge=ALL&crd_titre_recherche=&crd_auteur_merge=ALL&crd_auteur=&crd_affil_merge=ALL&crd_affil=&date=&publication_merge=ALL&publication=&crd_issn_merge=ALL&crd_issn=&crd_issn_match=EQ&id_number_merge=ALL&id_number=&id_number_match=EQ&crd_desc_mat=&crd_desc_mat2=&crd_desc_mat3=&crd_desc_geo=&crd_desc_geo2=&crd_desc_geo3=&crd_motscles_libres1_merge=ALL&crd_motscles_libres1=&crd_motscles_libres2_merge=ALL&crd_motscles_libres2=&subjects_merge=ANY&satisfyall=ALL&order=-dat

http://agritrop.cirad.fr/591119/
Fetch http://agritrop.cirad.fr/591119/ to ../tmp/team_6_addition\agritrop.cirad.fr\591119.html
sleep 1s
http://agritrop.cirad.fr/591372/
Fetch http://agritrop.cirad.fr/591372/ to ../tmp/team_6_addition\agritrop.cirad.fr\591372.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/591798/
Fetch http://agritrop.cirad.fr/591798/ to ../tmp/team_6_addition\agritrop.cirad.fr\591798.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/588701/
Fetch http://agritrop.cirad.fr/588701/ to ../tmp/team_6_addition\agritrop.cirad.fr\588701.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/590160/
Fetch http://agritrop.cirad.fr/590160/ to ../tmp/team_6_addition\agritrop.cirad.fr\590160.html
sleep 1s
http://agritrop.cirad.fr/591822/
Fetch http://agritrop.cirad.fr/591822/ to ../tmp/team_6_addition\agritrop.cirad.fr\591822.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/591744/
Fetch http://agritrop.cirad.fr/591744/ to ../

http://agritrop.cirad.fr/591623/
Fetch http://agritrop.cirad.fr/591623/ to ../tmp/team_6_addition\agritrop.cirad.fr\591623.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/590108/
Fetch http://agritrop.cirad.fr/590108/ to ../tmp/team_6_addition\agritrop.cirad.fr\590108.html
sleep 1s
http://agritrop.cirad.fr/592386/
Fetch http://agritrop.cirad.fr/592386/ to ../tmp/team_6_addition\agritrop.cirad.fr\592386.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/592541/
Fetch http://agritrop.cirad.fr/592541/ to ../tmp/team_6_addition\agritrop.cirad.fr\592541.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/592205/
Fetch http://agritrop.cirad.fr/592205/ to ../tmp/team_6_addition\agritrop.cirad.fr\592205.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/590980/
Fetch http://agritrop.cirad.fr/590980/ to ../tmp/team_6_addition\agritrop.cirad.fr\590980.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/590282/
Fetch http://agritro

http://agritrop.cirad.fr/591795/
Fetch http://agritrop.cirad.fr/591795/ to ../tmp/team_6_addition\agritrop.cirad.fr\591795.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/592556/
Fetch http://agritrop.cirad.fr/592556/ to ../tmp/team_6_addition\agritrop.cirad.fr\592556.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/592408/
Fetch http://agritrop.cirad.fr/592408/ to ../tmp/team_6_addition\agritrop.cirad.fr\592408.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/591414/
Fetch http://agritrop.cirad.fr/591414/ to ../tmp/team_6_addition\agritrop.cirad.fr\591414.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/590932/
Fetch http://agritrop.cirad.fr/590932/ to ../tmp/team_6_addition\agritrop.cirad.fr\590932.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/591276/
Fetch http://agritrop.cirad.fr/591276/ to ../tmp/team_6_addition\agritrop.cirad.fr\591276.html
sleep 1s
http://agritrop.cirad.fr/591887/
Fetch http://agritro

http://agritrop.cirad.fr/588816/
Fetch http://agritrop.cirad.fr/588816/ to ../tmp/team_6_addition\agritrop.cirad.fr\588816.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/589719/
Fetch http://agritrop.cirad.fr/589719/ to ../tmp/team_6_addition\agritrop.cirad.fr\589719.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/588982/
Fetch http://agritrop.cirad.fr/588982/ to ../tmp/team_6_addition\agritrop.cirad.fr\588982.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/589342/
Fetch http://agritrop.cirad.fr/589342/ to ../tmp/team_6_addition\agritrop.cirad.fr\589342.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/587939/
Fetch http://agritrop.cirad.fr/587939/ to ../tmp/team_6_addition\agritrop.cirad.fr\587939.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/588545/
Fetch http://agritrop.cirad.fr/588545/ to ../tmp/team_6_addition\agritrop.cirad.fr\588545.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/591

...

http://agritrop.cirad.fr/573349/
Fetch http://agritrop.cirad.fr/573349/ to ../tmp/team_6_addition\agritrop.cirad.fr\573349.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/573877/
Fetch http://agritrop.cirad.fr/573877/ to ../tmp/team_6_addition\agritrop.cirad.fr\573877.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/571827/
Fetch http://agritrop.cirad.fr/571827/ to ../tmp/team_6_addition\agritrop.cirad.fr\571827.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/569253/
Fetch http://agritrop.cirad.fr/569253/ to ../tmp/team_6_addition\agritrop.cirad.fr\569253.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/569799/
Fetch http://agritrop.cirad.fr/569799/ to ../tmp/team_6_addition\agritrop.cirad.fr\569799.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/567258/
Fetch http://agritrop.cirad.fr/567258/ to ../tmp/team_6_addition\agritrop.cirad.fr\567258.html
Already exists. Skipped.
sleep 1s
sleep 3s


Page 161
Fetch ht

http://agritrop.cirad.fr/569302/
Fetch http://agritrop.cirad.fr/569302/ to ../tmp/team_6_addition\agritrop.cirad.fr\569302.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/569183/
Fetch http://agritrop.cirad.fr/569183/ to ../tmp/team_6_addition\agritrop.cirad.fr\569183.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/579487/
Fetch http://agritrop.cirad.fr/579487/ to ../tmp/team_6_addition\agritrop.cirad.fr\579487.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/570573/
Fetch http://agritrop.cirad.fr/570573/ to ../tmp/team_6_addition\agritrop.cirad.fr\570573.html
Already exists. Skipped.
sleep 1s
sleep 3s


Page 163
Fetch https://agritrop.cirad.fr/cgi/search/archive/advanced?screen=Search&dataset=archive&_action_search=Search&crd_mot_merge=ALL&crd_mot=small-scale+water&crd_mot_sujet_merge=ALL&crd_mot_sujet=&crd_titre_recherche_merge=ALL&crd_titre_recherche=&crd_auteur_merge=ALL&crd_auteur=&crd_affil_merge=ALL&crd_affil=&date=&publication_merg

http://agritrop.cirad.fr/564401/
Fetch http://agritrop.cirad.fr/564401/ to ../tmp/team_6_addition\agritrop.cirad.fr\564401.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/570150/
Fetch http://agritrop.cirad.fr/570150/ to ../tmp/team_6_addition\agritrop.cirad.fr\570150.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/566643/
Fetch http://agritrop.cirad.fr/566643/ to ../tmp/team_6_addition\agritrop.cirad.fr\566643.html
Already exists. Skipped.
sleep 1s
sleep 3s


Page 165
Fetch https://agritrop.cirad.fr/cgi/search/archive/advanced?screen=Search&dataset=archive&_action_search=Search&crd_mot_merge=ALL&crd_mot=small-scale+water&crd_mot_sujet_merge=ALL&crd_mot_sujet=&crd_titre_recherche_merge=ALL&crd_titre_recherche=&crd_auteur_merge=ALL&crd_auteur=&crd_affil_merge=ALL&crd_affil=&date=&publication_merge=ALL&publication=&crd_issn_merge=ALL&crd_issn=&crd_issn_match=EQ&id_number_merge=ALL&id_number=&id_number_match=EQ&crd_desc_mat=&crd_desc_mat2=&crd_desc_mat3=&cr

http://agritrop.cirad.fr/563467/
Fetch http://agritrop.cirad.fr/563467/ to ../tmp/team_6_addition\agritrop.cirad.fr\563467.html
Already exists. Skipped.
sleep 1s
sleep 3s


Page 167
Fetch https://agritrop.cirad.fr/cgi/search/archive/advanced?screen=Search&dataset=archive&_action_search=Search&crd_mot_merge=ALL&crd_mot=small-scale+water&crd_mot_sujet_merge=ALL&crd_mot_sujet=&crd_titre_recherche_merge=ALL&crd_titre_recherche=&crd_auteur_merge=ALL&crd_auteur=&crd_affil_merge=ALL&crd_affil=&date=&publication_merge=ALL&publication=&crd_issn_merge=ALL&crd_issn=&crd_issn_match=EQ&id_number_merge=ALL&id_number=&id_number_match=EQ&crd_desc_mat=&crd_desc_mat2=&crd_desc_mat3=&crd_desc_geo=&crd_desc_geo2=&crd_desc_geo3=&crd_motscles_libres1_merge=ALL&crd_motscles_libres1=&crd_motscles_libres2_merge=ALL&crd_motscles_libres2=&subjects_merge=ANY&satisfyall=ALL&order=-date%2Fcreators_name%2Ftitle&search_offset=3340
http://agritrop.cirad.fr/566614/
Fetch http://agritrop.cirad.fr/566614/ to ../tmp/team_

http://agritrop.cirad.fr/563383/
Fetch http://agritrop.cirad.fr/563383/ to ../tmp/team_6_addition\agritrop.cirad.fr\563383.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/564112/
Fetch http://agritrop.cirad.fr/564112/ to ../tmp/team_6_addition\agritrop.cirad.fr\564112.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/563635/
Fetch http://agritrop.cirad.fr/563635/ to ../tmp/team_6_addition\agritrop.cirad.fr\563635.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/569003/
Fetch http://agritrop.cirad.fr/569003/ to ../tmp/team_6_addition\agritrop.cirad.fr\569003.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/565088/
Fetch http://agritrop.cirad.fr/565088/ to ../tmp/team_6_addition\agritrop.cirad.fr\565088.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/577013/
Fetch http://agritrop.cirad.fr/577013/ to ../tmp/team_6_addition\agritrop.cirad.fr\577013.html
Already exists. Skipped.
sleep 1s
http://agritrop.cirad.fr/567

In [121]:
import sys
from importlib import reload
sys.path.append('../src')
from webscrapping import webcrawler_main
webcrawler_main = reload(webcrawler_main)

_webcrawler_main = webcrawler_main.WebcrawlerMain()